In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.table import Table

In [171]:
def generate_connection_matrix(num_squares): 

    connection_matrix = np.zeros((num_squares, num_squares))
    nums = list(range(1, num_squares + 1))
    
    connection_matrix[0] = nums # creating 1st row
    for row in nums:
        for col in nums:
            allowed_nums = list(range(1, num_squares + 1))
            if connection_matrix[row - 1][col - 1] == 0: 
                # Checking the numbers already in this row and removing them from allowed nums
                for num in connection_matrix[row - 1]:
                    if num in allowed_nums: allowed_nums.remove(num)
                
                # Checking the numbers already in this column
                for num in [connection_matrix[_][col - 1] for _ in range(num_squares)]:
                    if num in allowed_nums: allowed_nums.remove(num)
                
                try: 
                    connection_matrix[row - 1][col - 1] = allowed_nums[0]
                except:
                    print(row,col)
                

    return connection_matrix.astype(int)
mat_4 = generate_connection_matrix(4)
mat_64 = generate_connection_matrix(64)



In [ ]:
def generate_hypercube_vertices(dimension):
    vertices = []
    def genbin(n, bs=''):
        if len(bs) == n:
            vertices.append(bs)
        else:
            genbin(n, bs + '0')
            genbin(n, bs + '1')

    genbin(dimension)
    return vertices
#generate_hypercube_vertices(4)

['0000',
 '0001',
 '0010',
 '0011',
 '0100',
 '0101',
 '0110',
 '0111',
 '1000',
 '1001',
 '1010',
 '1011',
 '1100',
 '1101',
 '1110',
 '1111']

In [ ]:
def find_1s(binary_string):
    binary_list = list(binary_string)
    idxs = []
    for char in binary_list:
        if char == "1":
            idx = binary_list.index(char) + 1
            idxs.append(idx)
            binary_list[idx - 1] = 0
    return idxs
#find_1s("0111")

[2, 3, 4]

In [ ]:
def colour_vertices(connection_matrix):
    dimension = len(connection_matrix)
    vertices = generate_hypercube_vertices(dimension)
    colours = []

    for vertex in vertices:
        flipped_dimensions = find_1s(vertex)
        colour = 1
        for dim in flipped_dimensions:
            new_colour = int(connection_matrix[dim - 1][colour - 1])
            colour = new_colour
        colours.append(colour)
    
    hypercube = {}
    n = 0 
    for vertex in vertices:
        hypercube[vertex] = colours[n]
        n+=1
    return hypercube
#colour_vertices(4)

In [ ]:
def find_colour(coins_string, connection_matrix):
    flipped_dimensions = find_1s(coins_string)
    colour = 1
    for dim in flipped_dimensions:
        new_colour = int(connection_matrix[dim-1][colour-1])
        colour = new_colour
    return colour
#find_colour("1111", mat_4)

In [243]:
def flip_this_coin(key_position, coins_string, connection_matrix):
    current_colour = find_colour(coins_string, connection_matrix)
    #print(current_colour)
    #print(connection_matrix)
    column = [int(connection_matrix[_][current_colour - 1]) for _ in range(len(connection_matrix))]
    #print(column)
    flip_dimension = column.index(key_position) + 1 

    return flip_dimension 
#flip_this_coin(3,"0010",mat_4)

In [ ]:
def flip_nth_bit(n, binary_string):
    binary_list = list(binary_string)
    if binary_list[n-1] == '0':
        binary_list[n-1] = '1'
    else:
        binary_list[n-1] = '0'
    return ''.join(binary_list)
#flip_nth_bit(4,'100001')

'100101'

In [239]:
def create_board_from_coins_list(coins_string):
    coins_list = list(coins_string)
    num_squares = len(coins_list)
    side_length = int(np.sqrt(num_squares))
    board = []

    index = 0
    for _ in range(side_length):
        row = coins_list[index:index+side_length]  # Take the next n elements for the row
        board.append([int(_) for _ in row])
        index += side_length
    
    return board

heads_test = "110100011"
create_board_from_coins_list(heads_test)

[[1, 1, 0], [1, 0, 0], [0, 1, 1]]

In [244]:
num_squares = 4
coin_strings = generate_hypercube_vertices(num_squares)
matrix = generate_connection_matrix(num_squares)

fails = []
for key_position in range(num_squares):
    for coin_string in coin_strings:
        initial_board = create_board_from_coins_list(coin_string)
        print(initial_board, coin_string)
        flip_coin = flip_this_coin(key_position, coin_string, matrix)
        new_coin_list = flip_nth_bit(flip_coin, coin_string)
        new_board = create_board_from_coins_list(new_coin_list)
        new_colour = find_colour(new_board)
        print(f'key = {key_position}', f'initial board = {initial_board}', 
              f'flip {flip_coin}', f'new_board {new_board}', 
              f'new_colour {new_colour}')
        if new_colour != key_position:
            fails.append(key_position, coin_strings)

fails

[[0, 0], [0, 0]] 0000


ValueError: 0 is not in list

In [ ]:
# Plotting

# Generate a sample 64x64 matrix (you can replace this with your actual matrix)
matrix = mat_64

# Create a figure and axis
fig, ax = plt.subplots(figsize=(11.7, 8.3))  # A4 size in inches (landscape)

# Hide the axes
ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)
ax.set_frame_on(False)

# Create a table
table = Table(ax, bbox=[0, 0, 1, 1])

# Add data to the table
nrows, ncols = matrix.shape
for i in range(nrows):
    for j in range(ncols):
        table.add_cell(i, j, 1/ncols, 1/nrows, text=f'{matrix[i, j]}', loc='center', edgecolor='black')

# Add the table to the plot
ax.add_table(table)

# Save the figure to a PDF
plt.savefig('matrix.pdf', bbox_inches='tight', pad_inches=0.1, dpi=300)

plt.show()
